# Crossovers

You've defined an AOI, you've specified the image types you are interested and the search query. Great! But are there crossovers between the image types? e.g. how many PSOrthotiles were collected within 1 day of a Landsat 8 scene?

In this notebook, we will find that answer.

In [90]:
# Notebook dependencies
from __future__ import print_function

import datetime
# import copy
# from functools import partial
import json
import os

import ipyleaflet as ipyl
import ipywidgets as ipyw
from IPython.display import display
# import matplotlib
# from matplotlib import cm
# import matplotlib.pyplot as plt
# import numpy as np
import pandas as pd
from planet import api
from planet.api import filters
# import pyproj
# import rasterio
# from rasterio import features as rfeatures
# from shapely import geometry as sgeom
# import shapely.ops

# %matplotlib inline

## Define AOI

Define the AOI as a geojson polygon. This can be done at [geojson.io](http://geojson.io). If you use geojson.io, only copy the single aoi feature, not the entire feature collection.

In [9]:
aoi = {u'geometry': {u'type': u'Polygon', u'coordinates': [[[-121.3113248348236, 38.28911976564886], [-121.3113248348236, 38.34622533958], [-121.2344205379486, 38.34622533958], [-121.2344205379486, 38.28911976564886], [-121.3113248348236, 38.28911976564886]]]}, u'type': u'Feature', u'properties': {u'style': {u'opacity': 0.5, u'fillOpacity': 0.2, u'noClip': False, u'weight': 4, u'color': u'blue', u'lineCap': None, u'dashArray': None, u'smoothFactor': 1, u'stroke': True, u'fillColor': None, u'clickable': True, u'lineJoin': None, u'fill': True}}}

In [10]:
json.dumps(aoi)

'{"geometry": {"type": "Polygon", "coordinates": [[[-121.3113248348236, 38.28911976564886], [-121.3113248348236, 38.34622533958], [-121.2344205379486, 38.34622533958], [-121.2344205379486, 38.28911976564886], [-121.3113248348236, 38.28911976564886]]]}, "type": "Feature", "properties": {"style": {"opacity": 0.5, "noClip": false, "weight": 4, "color": "blue", "lineCap": null, "smoothFactor": 1, "stroke": true, "fillOpacity": 0.2, "clickable": true, "fill": true, "dashArray": null, "fillColor": null, "lineJoin": null}}}'

## Build Request

Build the Planet API Filter request.

In [11]:
# filters.build_search_request() item types:
# Landsat 8 - 'Landsat8L1G'
# Sentinel - 'Sentinel2L1C'
# PS Orthotile = 'PSOrthoTile'

def build_landsat_request(aoi_geom, start_date, stop_date):
    # need to add filter for quality_category to only get L1T day scenes
    # with all the assets
    query = filters.and_filter(
        filters.geom_filter(aoi_geom),
        filters.range_filter('cloud_cover', lt=5),
        # ensure has all assets, unfortunately also filters 'L1TP'
#         filters.string_filter('quality_category', 'standard'), 
        filters.range_filter('sun_elevation', gt=0), # filter out night scenes
        filters.date_range('acquired', gt=start_date),
        filters.date_range('acquired', lt=stop_date)
    )

    return filters.build_search_request(query, ['Landsat8L1G'])    
    
    
def build_ps_request(aoi_geom, start_date, stop_date):
    query = filters.and_filter(
        filters.geom_filter(aoi_geom),
        filters.range_filter('cloud_cover', lt=5),
        filters.date_range('acquired', gt=start_date),
        filters.date_range('acquired', lt=stop_date)
    )

    return filters.build_search_request(query, ['PSOrthoTile'])


start_date = datetime.datetime(year=2017,month=1,day=1)
stop_date = datetime.datetime(year=2017,month=8,day=23)

print(build_landsat_request(aoi['geometry'], start_date, stop_date))
print(build_ps_request(aoi['geometry'], start_date, stop_date))

{'filter': {'type': 'AndFilter', 'config': ({'config': {u'type': u'Polygon', u'coordinates': [[[-121.3113248348236, 38.28911976564886], [-121.3113248348236, 38.34622533958], [-121.2344205379486, 38.34622533958], [-121.2344205379486, 38.28911976564886], [-121.3113248348236, 38.28911976564886]]]}, 'field_name': 'geometry', 'type': 'GeometryFilter'}, {'config': {'lt': 5}, 'field_name': 'cloud_cover', 'type': 'RangeFilter'}, {'config': {'gt': 0}, 'field_name': 'sun_elevation', 'type': 'RangeFilter'}, {'config': {'gt': '2017-01-01T00:00:00Z'}, 'field_name': 'acquired', 'type': 'DateRangeFilter'}, {'config': {'lt': '2017-08-23T00:00:00Z'}, 'field_name': 'acquired', 'type': 'DateRangeFilter'})}, 'item_types': ['Landsat8L1G']}
{'filter': {'type': 'AndFilter', 'config': ({'config': {u'type': u'Polygon', u'coordinates': [[[-121.3113248348236, 38.28911976564886], [-121.3113248348236, 38.34622533958], [-121.2344205379486, 38.34622533958], [-121.2344205379486, 38.28911976564886], [-121.311324834823

## Search Planet API

The client is how we interact with the planet api. It is created with the user-specific api key, which is pulled from $PL_API_KEY environment variable.

In [12]:
def get_api_key():
    return os.environ['PL_API_KEY']

# quick check that key is defined
assert get_api_key(), "PL_API_KEY not defined."

In [13]:
def create_client():
    return api.ClientV1(api_key=get_api_key())

def search_pl_api(request, limit=500):
    client = create_client()
    result = client.quick_search(request)
    
    # note that this returns a generator
    return result.items_iter(limit=limit)


items = list(search_pl_api(build_ps_request(aoi['geometry'], start_date, stop_date)))
print(len(items))
# uncomment below to see entire metadata for a PS orthotile
# print(json.dumps(items[0], indent=4))
del items

items = list(search_pl_api(build_landsat_request(aoi['geometry'], start_date, stop_date)))
print(len(items))
# uncomment below to see entire metadata for a landsat scene
# print(json.dumps(items[0], indent=4))
del items

133
42


In processing the items to scenes, we are only using a small subset of the [product metadata](https://www.planet.com/docs/spec-sheets/sat-imagery/#product-metadata). 

In [148]:
def items_to_scenes(items):
    item_types = []

    def _get_props(item):
        props = item['properties']
        props.update({
            'thumbnail': item['_links']['thumbnail'],
            'item_type': item['properties']['item_type'],
            'id': item['id'],
            'acquired': item['properties']['acquired'],
            'footprint': item['geometry']
        })
        return props
    
    scenes = pd.DataFrame(data=[_get_props(i) for i in items])
    
    # acquired column to index, for faster processing
    scenes.index = pd.to_datetime(scenes['acquired'])
    del scenes['acquired']
    scenes.sort_index(inplace=True)
    
    return scenes

scenes = items_to_scenes(search_pl_api(build_landsat_request(aoi['geometry'], start_date, stop_date)))
display(scenes[:1])
del scenes

,anomalous_pixels,cloud_cover,collection,columns,data_type,epsg_code,footprint,gsd,id,instrument,...,rows,satellite_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle,wrs_path,wrs_row
acquired,,,,,,,,,,,,,,,,,,,,,
2017-01-05 18:45:52.748510,0,0.15,NaN,7671,L1T,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",30,LC80440332017005LGN00,OLI_TIRS,...,7801,Landsat8,157.9,25.3,https://api.planet.com/data/v1/item-types/Land...,2017-04-26T14:09:25Z,0.68,0,44,33


## Investigate Landsat Scenes

In [152]:
landsat_scenes = items_to_scenes(search_pl_api(build_landsat_request(aoi['geometry'], start_date, stop_date)))
print(len(landsat_scenes))

42


In [153]:
def landsat_scenes_to_features_layer(scenes):
    features_style = {
            'color': 'grey',
            'weight': 1,
            'fillColor': 'grey',
            'fillOpacity': 0.15}

    features = [{"geometry": r.footprint,
                 "type": "Feature",
                 "properties": {"style": features_style,
                                "wrs_path": r.wrs_path,
                                "wrs_row": r.wrs_row}}
                for r in scenes.itertuples()]
    return features

def create_landsat_hover_handler(scenes):
    def hover_handler(event=None, id=None, properties=None):
        wrs_path = properties['wrs_path']
        wrs_row = properties['wrs_row']
        path_row_query = 'wrs_path=={} and wrs_row=={}'.format(wrs_path, wrs_row)
        count = len(scenes.query(path_row_query))
        label.value = 'path: {}, row: {}, count: {}'.format(wrs_path, wrs_row, count)
    return hover_handler


def create_landsat_feature_layer(scenes):
    
    features = landsat_scenes_to_features_layer(scenes)
    
    # Footprint feature layer
    feature_collection = {
        "type": "FeatureCollection",
        "features": features
    }

    feature_layer = ipyl.GeoJSON(data=feature_collection)

    label = ipyw.Label(layout=ipyw.Layout(width='100%'))

    feature_layer.on_hover(create_landsat_hover_handler(scenes))
    return feature_layer

In [154]:
# Initialize map using parameters from above map
# and deleting map instance if it exists
try:
    del fp_map
except NameError:
    pass


zoom = 6
center = [38.28993659801203, -120.14648437499999] # lat/lon

In [155]:
# Create map, adding box drawing controls
# Reuse parameters if map already exists
try:
    center = fp_map.center
    zoom = fp_map.zoom
    print(zoom)
    print(center)
except NameError:
    pass

# Change tile layer to one that makes it easier to see crop features
# Layer selected using https://leaflet-extras.github.io/leaflet-providers/preview/
map_tiles = ipyl.TileLayer(url='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png')
fp_map = ipyl.Map(
        center=center, 
        zoom=zoom,
        default_tiles = map_tiles
    )

fp_map.add_layer(create_landsat_feature_layer(landsat_scenes)) # landsat layer
fp_map.add_layer(ipyl.GeoJSON(data=aoi)) # aoi layer
    
# Display map and label
ipyw.VBox([fp_map, label])

This AOI is located in a region covered by 3 different path/row tiles. This means there is 3x the coverage than in regions only covered by one path/row tile. This is particularly lucky!

What about the within each path/row tile. How long and how consistent is the Landsat 8 collect period for each path/row?

In [151]:
def time_diff_stats(group):
    time_diff = group.index.to_series().diff() # time difference between rows in group
    stats = {'median': time_diff.median(),
             'mean': time_diff.mean(),
             'std': time_diff.std(),
             'count': time_diff.count(),
             'min': time_diff.min(),
             'max': time_diff.max()}
    return pd.Series(stats)

landsat_scenes.groupby(['wrs_path', 'wrs_row']).apply(time_diff_stats)

count                      max                     mean  \
wrs_path wrs_row                                                            
43       33          12  31 days 23:59:46.812643  17 days 07:59:59.313376   
         34          13  16 days 00:00:11.536172  15 days 23:59:59.366519   
44       33          14  16 days 00:00:10.236295  15 days 23:59:59.404161   

                                   median                      min  \
wrs_path wrs_row                                                     
43       33       16 days 00:00:04.366736  15 days 23:59:50.258638   
         34       15 days 23:59:55.543541  15 days 23:59:50.262876   
44       33       15 days 23:59:59.511409  15 days 23:59:49.212960   

                                     std  
wrs_path wrs_row                          
43       33       4 days 14:51:00.569425  
         34       0 days 00:00:07.696147  
44       33       0 days 00:00:07.579903

It looks like the collection period is just 10seconds short of 16 days. 

path/row 43/33 is missing one image which causes an unusually long collect period.

What this means is that we don't need to look at every Landsat 8 scene collect time to find crossovers with Planet scenes. We could look at the first scene for each path/row, then look at every 16 day increment. However, we will need to account for dropped Landsat 8 scenes in some way.

What is the time difference between the tiles?

In [181]:
def find_closest(date_time, data_frame):
    time_deltas = (data_frame.index - date_time).to_series().reset_index(drop=True).abs()
    idx_min = time_deltas.idxmin()

    min_delta = time_deltas[idx_min]
    return (idx_min, min_delta)

def closest_time(group):
    inquiry_date = datetime.datetime(year=2017,month=3,day=7)
    idx, _ = find_closest(inquiry_date, group)
    return group.index.to_series().iloc[idx]


landsat_scenes.groupby(['wrs_path', 'wrs_row']).apply(closest_time)

wrs_path  wrs_row
43        33        2017-03-03 18:39:20.127296
          34        2017-03-03 18:39:44.009861
44        33        2017-03-10 18:45:27.068563
dtype: datetime64[ns]

So the tiles that are in the same path are very close (20sec) together, basically from the same day. Therefore, we would want to only use one tile and pick the best image.

Tiles that are in different paths are 7 days apart. Therefore, we want to keep tiles from different paths, as they represent unique crossovers.

## Investigate PS Scenes

In [182]:
ps_scenes = items_to_scenes(search_pl_api(build_ps_request(aoi['geometry'], start_date, stop_date)))
print(len(ps_scenes))
display(ps_scenes[:1])

133


,anomalous_pixels,black_fill,cloud_cover,columns,epsg_code,footprint,grid_cell,ground_control,gsd,id,...,published,rows,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle
acquired,,,,,,,,,,,,,,,,,,,,,
2017-01-21 18:06:20.982805,0.24,0.743849,0.239,8000,32610,"{u'type': u'Polygon', u'coordinates': [[[-121....",1056721,NaN,3.967036,371816_1056721_2017-01-21_0e30,...,2017-01-22T05:48:31Z,8000,0e30,371816,146.1,24.4,https://api.planet.com/data/v1/item-types/PSOr...,2017-01-30T12:17:20Z,0.02,0.1


Ideally, PS scenes have daily coverage over all regions. We want to identify images that occur on the same day and only select the best ones. 

In [215]:
daily_ps_scenes = ps_scenes.index.to_series().groupby([ps_scenes.index.year,
                                                       ps_scenes.index.month,
                                                       ps_scenes.index.day])

In [216]:
daily_count = daily_ps_scenes.agg('count')
daily_count[daily_count > 1]

2017  4  20    2
         29    2
      5  7     2
         18    2
         21    2
      6  14    2
         16    2
         17    3
         22    3
         27    3
      7  1     2
         2     3
         4     3
         6     2
         7     2
         8     2
         10    2
         11    2
         13    3
         17    2
         20    3
         21    2
         26    2
         27    2
         28    2
         29    2
         31    2
      8  2     2
         7     3
         8     2
         15    3
         21    2
Name: acquired, dtype: int64

In [227]:
def scenes_and_count(group):
    entry = {'count': len(group),
             'scenes': group.index.tolist()}
    
    return pd.DataFrame(entry)

daily_count_and_scenes = daily_ps_scenes.apply(scenes_and_count)
multiplecoverage = daily_count_and_scenes.query('ilevel_1 == 4 and count > 1')
multiplecoverage

count                     scenes
2017 4 20 0      2 2017-04-20 18:01:32.425201
          1      2 2017-04-20 18:09:59.004133
       29 0      2 2017-04-29 18:01:16.191189
          1      2 2017-04-29 18:02:17.982774

## Find Crossovers

Use pandas magic to break scenes up by types (`item_type`), then find matching days

In [ ]:
# https://stackoverflow.com/questions/36933725/pandas-time-series-join-by-closest-time
def find_closest(aquired_time, data_frame):
    time_deltas = (data_frame.index - aquired_time).to_series().reset_index(drop=True).abs()
    idx_min = time_deltas.idxmin()

    min_delta = time_deltas[idx_min]
    return (idx_min, min_delta)


def find_crossovers(aquired_time):
    closest_idx, closest_delta = find_closest(aquired_time, landsat_scenes)
    closest_landsat = landsat_scenes.iloc[idx_min]

    crossover = {'landsat_acquisition': closest_landsat.name,
                 'delta': min_delta}
    return pd.Series(crossover)

ps_scenes = items_to_scenes(search_pl_api(build_ps_request(aoi['geometry'], start_date, stop_date)))

# apply closest_landsat_info() to each row
crossovers = ps_scenes.index.to_series().apply(find_closest)
crossovers[crossovers['delta'] < pd.Timedelta('1 days')]

In [147]:
# https://stackoverflow.com/questions/36933725/pandas-time-series-join-by-closest-time
def find_closest(ps_datetime):
    time_deltas = (landsat_scenes.index - ps_datetime).to_series().reset_index(drop=True).abs()
    idx_min = time_deltas.idxmin()

    min_delta = time_deltas[idx_min]

    closest_landsat = landsat_scenes.iloc[idx_min]

    crossover = {'landsat_acquisition': closest_landsat.name,
                 'delta': min_delta}
    return pd.Series(crossover)

ps_scenes = items_to_scenes(search_pl_api(build_ps_request(aoi['geometry'], start_date, stop_date)))

# apply closest_landsat_info() to each row
crossovers = ps_scenes.index.to_series().apply(find_closest)
crossovers[crossovers['delta'] < pd.Timedelta('1 days')]

,delta,landsat_acquisition
acquired,,
2017-01-21 18:06:20.982805,00:39:27.144472,2017-01-21 18:45:48.127277
2017-01-30 18:07:08.365512,00:32:24.949141,2017-01-30 18:39:33.314653
2017-04-20 18:01:32.425201,00:37:21.460219,2017-04-20 18:38:53.885420
2017-04-20 18:09:59.004133,00:28:54.881287,2017-04-20 18:38:53.885420
2017-04-27 18:03:00.275770,00:41:59.185523,2017-04-27 18:44:59.461293
2017-05-06 18:08:35.673239,00:30:13.751485,2017-05-06 18:38:49.424724
2017-05-07 18:02:14.522227,23:23:01.206464,2017-05-06 18:39:13.315763
2017-05-07 18:02:49.753154,23:23:36.437391,2017-05-06 18:39:13.315763
2017-05-13 18:02:27.453337,00:42:38.184447,2017-05-13 18:45:05.637784
